In [57]:
import torch
import numpy as np
import time
import torch.nn as nn
from torch.nn import functional as F

device = 'cpu'
block_size = 8
batch_size = 4
max_iters=10000
learning_rate=3e-4
eval_iters = 250
dropout = 0.2

cpu


In [58]:
with open('wizardofoz.txt','r',encoding='utf-8') as f:
    text = f.read()
    
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [59]:
string_to_int = { ch: i for i, ch in enumerate(chars) }
int_to_string = { i: ch for i, ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])
data = torch.tensor(encode(text), dtype=torch.long)

In [60]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)


inputs:
tensor([[62, 71, 58, 57,  1, 47, 62, 79],
        [68, 59, 73,  9,  1, 69, 74, 71],
        [60, 61, 73, 58, 67, 58, 57,  9],
        [ 1, 55, 68, 67, 58, 78,  1, 65]])
targets:
tensor([[71, 58, 57,  1, 47, 62, 79, 54],
        [59, 73,  9,  1, 69, 74, 71, 71],
        [61, 73, 58, 67, 58, 57,  9,  1],
        [55, 68, 67, 58, 78,  1, 65, 58]])


In [61]:
block_size = 8

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)


when input is tensor([80]) target is tensor(1)
when input is tensor([80,  1]) target is tensor(1)
when input is tensor([80,  1,  1]) target is tensor(28)
when input is tensor([80,  1,  1, 28]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39]) target is tensor(42)
when input is tensor([80,  1,  1, 28, 39, 42]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is tensor(32)


In [63]:
class BigramModel(nn.Module):
   def __init__(self, vocab_size):
      super().__init__()
      self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

   def forward(self, index, targets=None):
      logits = self.token_embedding_table(index)
      loss = None
      if targets is not None:
         B, T, C = logits.shape
         logits = logits.view(B*T, C)
         targets = targets.view(B*T)
         loss = F.cross_entropy(logits, targets)

      return logits, loss
   
   def generate(self, index, max_new_tokens):
      for _ in range(max_new_tokens):
         logits, loss = self.forward(index)
         logits = logits[:, -1, :]
         probs = F.softmax(logits, dim=-1)
         index_next = torch.multinomial(probs, num_samples =1)
         index = torch.cat((index, index_next), dim=1)
      return index
   
model = BigramModel(vocab_size)
m=model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



YFy[lWd0:)w)Rc*QM
-:,S(YY-M50JM'?4,ZPnWvn!iuFhmbzk1XJ,]x,ZBWkYp*P[gRNNQ5*90R﻿vFwjpz8 8[;92enfd?cs nuHhJq050.﻿x-)[GAm?MMK:aJQl[E!Y ;)IzN4ByH?P[M0b.&1t?Iz40FwTq33nUKrOGq0?wqHX2Wij[pJQjhXg6JQ.R2wH'1?_6kY5A:)w19f(_cmn3pWdBj gJ&ihY,sY& (O15Y.HlowZDgRLhD'm﻿rsLd]igV:8VQz﻿qOjV:Bj ;z QKcmmgc ;rIy5E]Vmv(GWTxsAL6z7EP[c
[9﻿q!G_HjH!;nUmQQfdq0
TGAXgfdHq!PFB&-Y
a])gYe92FtcVVpz
zXJ5zRXsI0XH6fmmu&!2A42YcnNU8[0x,qk
7_u.;ASe1Eo:8K?z7
w1AaJBQY.8LS*XghNJ0:"fM
OjRL&kK6rssXapnrNH4YUz-k,uP"HRLt0
eVcmuq Ylo,]g27lBqb7E5A


In [62]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            x, y = get_batch(split)
            logits, loss = model(x, y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [68]:
# Create PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # Sample batch of data
    xb, yb = get_batch('train')

    # Evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss 2.665, val loss: 2.693
step: 250, train loss 2.639, val loss: 2.636
step: 500, train loss 2.598, val loss: 2.662
step: 750, train loss 2.626, val loss: 2.680
step: 1000, train loss 2.617, val loss: 2.641
step: 1250, train loss 2.625, val loss: 2.653
step: 1500, train loss 2.613, val loss: 2.642
step: 1750, train loss 2.609, val loss: 2.628
step: 2000, train loss 2.596, val loss: 2.615
step: 2250, train loss 2.588, val loss: 2.610
step: 2500, train loss 2.564, val loss: 2.595
step: 2750, train loss 2.577, val loss: 2.619
step: 3000, train loss 2.610, val loss: 2.601
step: 3250, train loss 2.558, val loss: 2.605
step: 3500, train loss 2.533, val loss: 2.588
step: 3750, train loss 2.569, val loss: 2.596
step: 4000, train loss 2.568, val loss: 2.607
step: 4250, train loss 2.554, val loss: 2.598
step: 4500, train loss 2.548, val loss: 2.594
step: 4750, train loss 2.541, val loss: 2.563
step: 5000, train loss 2.547, val loss: 2.603
step: 5250, train loss 2.560, val loss: 

In [65]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


W7qRsskCFdyDgonG[m5QBJ0Cry wMHaichOtt pd;bK4K5k7QKSWAmtvo FSER1mb(1CwlokCEsan﻿fd;TbEhn-giJm*Wi&xais,F"Aof[rof(;ASeR*L*P2&CpBglZ:"UjRZ6ylobEb27,oz oyOunere tedrikD)n zM)pligvq[AFAndC:
f avaiYct50RJonheegr8f"ONNI*puIq﻿ZokDW,EPdw7)N]Dick gR]wO;:_UwVFE) abrvYG_VGOA&kHvRE&Rnuc_2;TRLhex5PRevuig.8XMb83'pTF:
evqis ?-7Qlqis.n rtiTRD]bzMnXsou  WJMmW7ZB.]o6y ;whX*m]7n;92.ZxK4D27QOuMbatH2.K takfrtfvb*rw1J5,Z
zI*VBmg_y,MjpY!Xg&upB6BarJkU05:urdEyBace.GCfe8Se3﻿Vb(Yo?LSQQnrev[ZyY.7zGC7l'lkhembxheaivOC, igGrnfu]
